<a href="https://colab.research.google.com/github/priyanka2454/AML/blob/main/Assignment_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1.Make epochs=25 wherever it is
2.Model Summaries of each and every question


In [ ]:
from google.colab import files

files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"priyankateddy","key":"2cac501fc147aa1f2226979b8c352245"}'}

In [ ]:
!mkdir ~/.kaggle

!cp kaggle.json ~/.kaggle/

!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle competitions download -c dogs-vs-cats



 96% 783M/812M [00:03<00:00, 241MB/s]
100% 812M/812M [00:03<00:00, 237MB/s]


In [ ]:
!unzip -qq dogs-vs-cats.zip

!unzip -qq train.zip

In [ ]:
import os
import shutil

# Define split sizes as per question requirement
train_size = 500  # images per class
validation_size = 250  # images per class
test_size = 250  # images per class

# Source directory
source_dir = 'train'

# Target directories
base_dir = '/content/cats_and_dogs'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')

# Make directories if not present
os.makedirs(train_dir + '/cats', exist_ok=True)
os.makedirs(train_dir + '/dogs', exist_ok=True)
os.makedirs(validation_dir + '/cats', exist_ok=True)
os.makedirs(validation_dir + '/dogs', exist_ok=True)
os.makedirs(test_dir + '/cats', exist_ok=True)
os.makedirs(test_dir + '/dogs', exist_ok=True)

# Get lists of all images
all_images = os.listdir(source_dir)
cat_images = [img for img in all_images if 'cat' in img]
dog_images = [img for img in all_images if 'dog' in img]

# Copy images to target directories
def copy_images(images, start, end, target_dir):
    for img in images[start:end]:
        shutil.copy(os.path.join(source_dir, img), target_dir)

# Assign images to train, validation, and test sets
copy_images(cat_images, 0, train_size, os.path.join(train_dir, 'cats'))
copy_images(dog_images, 0, train_size, os.path.join(train_dir, 'dogs'))

copy_images(cat_images, train_size, train_size + validation_size, os.path.join(validation_dir, 'cats'))
copy_images(dog_images, train_size, train_size + validation_size, os.path.join(validation_dir, 'dogs'))

copy_images(cat_images, train_size + validation_size, train_size + validation_size + test_size, os.path.join(test_dir, 'cats'))
copy_images(dog_images, train_size + validation_size, train_size + validation_size + test_size, os.path.join(test_dir, 'dogs'))


In [ ]:
import os

# Directories
train_dir = '/content/cats_and_dogs/train'
validation_dir = '/content/cats_and_dogs/validation'
test_dir = '/content/cats_and_dogs/test'

# Function to count images in each class directory
def count_images(directory):
    cat_count = len(os.listdir(os.path.join(directory, 'cats')))
    dog_count = len(os.listdir(os.path.join(directory, 'dogs')))
    return cat_count, dog_count

# Verify training set
train_cat_count, train_dog_count = count_images(train_dir)
print(f"Training set: {train_cat_count} cats, {train_dog_count} dogs (Total: {train_cat_count + train_dog_count})")

# Verify validation set
val_cat_count, val_dog_count = count_images(validation_dir)
print(f"Validation set: {val_cat_count} cats, {val_dog_count} dogs (Total: {val_cat_count + val_dog_count})")

# Verify test set
test_cat_count, test_dog_count = count_images(test_dir)
print(f"Test set: {test_cat_count} cats, {test_dog_count} dogs (Total: {test_cat_count + test_dog_count})")


Training set: 500 cats, 500 dogs (Total: 1000)
Validation set: 250 cats, 250 dogs (Total: 500)
Test set: 250 cats, 250 dogs (Total: 500)


In [ ]:
#Question - 1
#Base Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Basic data preprocessing (rescaling only)
train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Directory paths
train_dir = '/content/cats_and_dogs/train'
validation_dir = '/content/cats_and_dogs/validation'
test_dir = '/content/cats_and_dogs/test'

# Data generators
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

# Simple CNN Model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2, 2),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),

    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),

    Flatten(),
    Dense(512, activation='relu'),
    Dense(1, activation='sigmoid')  # Sigmoid for binary classification
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model (baseline)
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=25,  # You can start with fewer epochs for initial testing
    validation_data=validation_generator,
    validation_steps=len(validation_generator)
)

# Evaluate on test data
test_loss, test_acc = model.evaluate(test_generator, steps=len(test_generator))
print(f"Test accuracy (Baseline): {test_acc}")


Found 1000 images belonging to 2 classes.
Found 500 images belonging to 2 classes.
Found 500 images belonging to 2 classes.
Epoch 1/25
32/32 [==============================] - 9s 241ms/step - loss: 0.8186 - accuracy: 0.5360 - val_loss: 0.7198 - val_accuracy: 0.5000
Epoch 2/25
32/32 [==============================] - 7s 233ms/step - loss: 0.6801 - accuracy: 0.5690 - val_loss: 0.6670 - val_accuracy: 0.5760
Epoch 3/25
32/32 [==============================] - 7s 230ms/step - loss: 0.6084 - accuracy: 0.6760 - val_loss: 0.6645 - val_accuracy: 0.6260
Epoch 4/25
32/32 [==============================] - 7s 226ms/step - loss: 0.5246 - accuracy: 0.7440 - val_loss: 0.6356 - val_accuracy: 0.6620
Epoch 5/25
32/32 [==============================] - 7s 233ms/step - loss: 0.4134 - accuracy: 0.8190 - val_loss: 0.6924 - val_accuracy: 0.6540
Epoch 6/25
32/32 [==============================] - 7s 230ms/step - loss: 0.3361 - accuracy: 0.8520 - val_loss: 0.7893 - val_accuracy: 0.6540
Epoch 7/25
32/32 [======

#Use any technique
to reduce overfitting and improve performance in developing a network that you train
from scratch. What performance did you achieve?

In [ ]:
# Data Augmentation for training
train_datagen_augmented = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Validation and test data remain the same (only rescaling)
validation_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Load augmented data for training
train_generator_augmented = train_datagen_augmented.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

# Train the model with augmented data
history_augmented = model.fit(
    train_generator_augmented,
    steps_per_epoch=len(train_generator_augmented),
    epochs=25,  # Train for the same number of epochs as baseline
    validation_data=validation_generator,
    validation_steps=len(validation_generator)
)

# Evaluate the model
test_loss_augmented, test_acc_augmented = model.evaluate(test_generator, steps=len(test_generator))
print(f"Test accuracy with Data Augmentation: {test_acc_augmented}")


Found 1000 images belonging to 2 classes.
Found 500 images belonging to 2 classes.
Found 500 images belonging to 2 classes.
Epoch 1/25
32/32 [==============================] - 9s 280ms/step - loss: 0.9361 - accuracy: 0.5410 - val_loss: 0.6813 - val_accuracy: 0.6360
Epoch 2/25
32/32 [==============================] - 9s 279ms/step - loss: 0.6837 - accuracy: 0.5520 - val_loss: 0.6545 - val_accuracy: 0.6180
Epoch 3/25
32/32 [==============================] - 9s 280ms/step - loss: 0.6684 - accuracy: 0.5910 - val_loss: 0.6441 - val_accuracy: 0.6520
Epoch 4/25
32/32 [==============================] - 9s 277ms/step - loss: 0.6589 - accuracy: 0.6280 - val_loss: 0.6336 - val_accuracy: 0.6500
Epoch 5/25
32/32 [==============================] - 9s 277ms/step - loss: 0.6337 - accuracy: 0.6320 - val_loss: 0.6106 - val_accuracy: 0.6880
Epoch 6/25
32/32 [==============================] - 9s 281ms/step - loss: 0.6302 - accuracy: 0.6310 - val_loss: 0.6284 - val_accuracy: 0.6620
Epoch 7/25
32/32 [======

In [ ]:
from tensorflow.keras.layers import Dropout

# Define the model with Dropout
model_with_dropout = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2, 2),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),

    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),

    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),  # Dropout to prevent overfitting
    Dense(1, activation='sigmoid')
])

# Compile the model
model_with_dropout.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model with dropout
history_dropout = model_with_dropout.fit(
    train_generator_augmented,
    steps_per_epoch=len(train_generator_augmented),
    epochs=25,
    validation_data=validation_generator,
    validation_steps=len(validation_generator)
)

# Evaluate the model
test_loss_dropout, test_acc_dropout = model_with_dropout.evaluate(test_generator, steps=len(test_generator))
print(f"Test accuracy with Data Augmentation and Dropout: {test_acc_dropout}")


Epoch 1/25
32/32 [==============================] - 10s 283ms/step - loss: 0.8168 - accuracy: 0.5040 - val_loss: 0.6905 - val_accuracy: 0.5620
Epoch 2/25
32/32 [==============================] - 9s 275ms/step - loss: 0.6887 - accuracy: 0.5490 - val_loss: 0.6805 - val_accuracy: 0.5840
Epoch 3/25
32/32 [==============================] - 9s 273ms/step - loss: 0.6801 - accuracy: 0.5680 - val_loss: 0.6768 - val_accuracy: 0.6280
Epoch 4/25
32/32 [==============================] - 9s 272ms/step - loss: 0.6841 - accuracy: 0.5470 - val_loss: 0.6704 - val_accuracy: 0.5840
Epoch 5/25
32/32 [==============================] - 9s 278ms/step - loss: 0.6751 - accuracy: 0.6030 - val_loss: 0.6987 - val_accuracy: 0.5440
Epoch 6/25
32/32 [==============================] - 9s 276ms/step - loss: 0.6775 - accuracy: 0.5920 - val_loss: 0.6583 - val_accuracy: 0.6100
Epoch 7/25
32/32 [==============================] - 9s 277ms/step - loss: 0.6561 - accuracy: 0.6150 - val_loss: 0.6728 - val_accuracy: 0.5880
Epoch

In [ ]:
from tensorflow.keras.layers import BatchNormalization

# Define the model with Batch Normalization
model_with_batchnorm = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    BatchNormalization(),
    MaxPooling2D(2, 2),

    Conv2D(64, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(2, 2),

    Conv2D(128, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(2, 2),

    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

# Compile the model
model_with_batchnorm.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model with batch normalization
history_batchnorm = model_with_batchnorm.fit(
    train_generator_augmented,
    steps_per_epoch=len(train_generator_augmented),
    epochs=25,
    validation_data=validation_generator,
    validation_steps=len(validation_generator)
)

# Evaluate the model
test_loss_batchnorm, test_acc_batchnorm = model_with_batchnorm.evaluate(test_generator, steps=len(test_generator))
print(f"Test accuracy with Data Augmentation, Dropout, and Batch Normalization: {test_acc_batchnorm}")


Epoch 1/25
32/32 [==============================] - 12s 329ms/step - loss: 7.1288 - accuracy: 0.5300 - val_loss: 4.4716 - val_accuracy: 0.5020
Epoch 2/25
32/32 [==============================] - 10s 315ms/step - loss: 3.4645 - accuracy: 0.5680 - val_loss: 7.5162 - val_accuracy: 0.5000
Epoch 3/25
32/32 [==============================] - 10s 308ms/step - loss: 1.3108 - accuracy: 0.5620 - val_loss: 7.1414 - val_accuracy: 0.4980
Epoch 4/25
32/32 [==============================] - 10s 322ms/step - loss: 0.7718 - accuracy: 0.6100 - val_loss: 7.6391 - val_accuracy: 0.5000
Epoch 5/25
32/32 [==============================] - 10s 314ms/step - loss: 0.7135 - accuracy: 0.6170 - val_loss: 7.9006 - val_accuracy: 0.5020
Epoch 6/25
32/32 [==============================] - 10s 311ms/step - loss: 0.7143 - accuracy: 0.6290 - val_loss: 4.0690 - val_accuracy: 0.5060
Epoch 7/25
32/32 [==============================] - 10s 312ms/step - loss: 0.6566 - accuracy: 0.6280 - val_loss: 2.7513 - val_accuracy: 0.5020

In [ ]:
from tensorflow.keras.callbacks import ReduceLROnPlateau

# Learning rate scheduler callback
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)

# Train the model with learning rate scheduler
history_final = model_with_batchnorm.fit(
    train_generator_augmented,
    steps_per_epoch=len(train_generator_augmented),
    epochs=25,  # Use more epochs to allow the scheduler to take effect
    validation_data=validation_generator,
    validation_steps=len(validation_generator),
    callbacks=[lr_scheduler]
)

# Evaluate the model
test_loss_final, test_acc_final = model_with_batchnorm.evaluate(test_generator, steps=len(test_generator))
print(f"Test accuracy with Data Augmentation, Dropout, Batch Normalization, and Learning Rate Scheduling: {test_acc_final}")


Epoch 1/25
32/32 [==============================] - 10s 315ms/step - loss: 0.5878 - accuracy: 0.6770 - val_loss: 0.7208 - val_accuracy: 0.5920 - lr: 0.0010
Epoch 2/25
32/32 [==============================] - 10s 307ms/step - loss: 0.5807 - accuracy: 0.7040 - val_loss: 0.6650 - val_accuracy: 0.7200 - lr: 0.0010
Epoch 3/25
32/32 [==============================] - 10s 307ms/step - loss: 0.5913 - accuracy: 0.6840 - val_loss: 0.6917 - val_accuracy: 0.6660 - lr: 0.0010
Epoch 4/25
32/32 [==============================] - 10s 307ms/step - loss: 0.5790 - accuracy: 0.6840 - val_loss: 0.6584 - val_accuracy: 0.6300 - lr: 0.0010
Epoch 5/25
32/32 [==============================] - 10s 310ms/step - loss: 0.5912 - accuracy: 0.7130 - val_loss: 0.6399 - val_accuracy: 0.6400 - lr: 0.0010
Epoch 6/25
32/32 [==============================] - 10s 310ms/step - loss: 0.5980 - accuracy: 0.6980 - val_loss: 0.6555 - val_accuracy: 0.6240 - lr: 0.0010
Epoch 7/25
32/32 [==============================] - 10s 309ms/st

I developed and optimized a neural network from scratch for the Cats & Dogs classification task. I started with a simple CNN model and gradually introduced various techniques to improve performance and reduce overfitting. Here’s a summary of each approach and its impact on test accuracy:

Baseline Model: The initial CNN model, trained with basic rescaling, achieved a test accuracy of 70.6%. This served as a starting point for further improvements.

Data Augmentation: By adding data augmentation techniques such as rotation, width/height shift, shear, zoom, and horizontal flip, I aimed to increase data variability. The model achieved a test accuracy of 70.2%—a slight drop, but the model became more robust.

Dropout: Introducing a 50% dropout layer after the dense layer helped reduce overfitting. With data augmentation and dropout, the model achieved a test accuracy of 70.8%, slightly improving stability.

Batch Normalization: Adding batch normalization layers after each convolutional layer further stabilized learning. With data augmentation, dropout, and batch normalization, the test accuracy dropped to 60.6%, indicating potential issues with model complexity or over-regularization.

Learning Rate Scheduling: Finally, I applied a learning rate scheduler (ReduceLROnPlateau) to adaptively decrease the learning rate based on validation loss. This setup achieved a test accuracy of 74.6%, the highest in this sequence, by allowing the model to fine-tune learning rates as training progressed.

Overall, through incremental improvements—data augmentation, dropout, batch normalization, and learning rate scheduling—I was able to develop a more robust model, achieving the best performance with a combination of these techniques, culminating in a 74.6% test accuracy.

In [ ]:
#Question 2
# Imports
import os
import shutil
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Step 1: Increase the Training Sample Size (1200 images per class)

# Define new training size (1200 images each for cats and dogs, 2400 in total)
new_train_size = 1200

# Set up paths
base_dir = '/content/cats_and_dogs'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')
source_dir = 'train'  # Assumes all images are in 'train_data' directory

# Clear and recreate training directories if they exist
if os.path.exists(train_dir):
    shutil.rmtree(train_dir)
os.makedirs(train_dir + '/cats', exist_ok=True)
os.makedirs(train_dir + '/dogs', exist_ok=True)

# Get lists of all images
all_images = os.listdir(source_dir)
cat_images = [img for img in all_images if 'cat' in img]
dog_images = [img for img in all_images if 'dog' in img]

# Function to copy images to the target directory
def copy_images(images, start, end, target_dir):
    for img in images[start:end]:
        shutil.copy(os.path.join(source_dir, img), target_dir)

# Copy 1200 images each for cats and dogs to the training directory
copy_images(cat_images, 0, new_train_size, os.path.join(train_dir, 'cats'))
copy_images(dog_images, 0, new_train_size, os.path.join(train_dir, 'dogs'))

# Step 2: Define and Train the Baseline Model from Scratch

# Basic data preprocessing (rescaling only, no augmentation yet)
train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Data generators for training, validation, and testing
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),      # Resize all images to 150x150 pixels
    batch_size=32,               # Number of images to yield per batch
    class_mode='binary'          # Binary classification (cats vs. dogs)
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

# Simple CNN Model (Baseline) without optimizations
baseline_model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2, 2),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),

    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),

    Flatten(),
    Dense(512, activation='relu'),
    Dense(1, activation='sigmoid')  # Sigmoid for binary classification
])

# Compile the baseline model
baseline_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the baseline model on the larger dataset (without optimizations)
history_baseline = baseline_model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),  # Total number of batches per epoch
    epochs=25,                             # Use fewer epochs for the baseline test
    validation_data=validation_generator,
    validation_steps=len(validation_generator)  # Total number of validation batches per epoch
)

# Evaluate the baseline model on the test data
test_loss_baseline, test_acc_baseline = baseline_model.evaluate(test_generator, steps=len(test_generator))
print(f"Test accuracy with increased training size (Baseline): {test_acc_baseline}")

# Step 3: Add Optimizations - Data Augmentation, Dropout, Batch Normalization

from tensorflow.keras.layers import Dropout, BatchNormalization

# Data Augmentation for the training set to prevent overfitting
train_datagen_augmented = ImageDataGenerator(
    rescale=1./255,             # Rescale pixel values from 0-255 to 0-1
    rotation_range=40,           # Randomly rotate images up to 40 degrees
    width_shift_range=0.2,       # Randomly shift images horizontally by 20%
    height_shift_range=0.2,      # Randomly shift images vertically by 20%
    shear_range=0.2,             # Apply shearing transformation
    zoom_range=0.2,              # Randomly zoom in on images by 20%
    horizontal_flip=True,        # Randomly flip images horizontally
    fill_mode='nearest'          # Fill in missing pixels with the nearest ones
)

# Load augmented data for training
train_generator_augmented = train_datagen_augmented.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

# Define the optimized model with Dropout and Batch Normalization
optimized_model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    BatchNormalization(),
    MaxPooling2D(2, 2),

    Conv2D(64, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(2, 2),

    Conv2D(128, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(2, 2),

    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),                # Dropout layer to reduce overfitting
    Dense(1, activation='sigmoid')
])

# Compile the optimized model
optimized_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the optimized model with augmented data
history_optimized = optimized_model.fit(
    train_generator_augmented,
    steps_per_epoch=len(train_generator_augmented),
    epochs=25,                   # More epochs to fully utilize the optimizations
    validation_data=validation_generator,
    validation_steps=len(validation_generator)
)

# Evaluate the optimized model on the test data
test_loss_optimized, test_acc_optimized = optimized_model.evaluate(test_generator, steps=len(test_generator))
print(f"Test accuracy with Data Augmentation, Dropout, and Batch Normalization: {test_acc_optimized}")


Found 2400 images belonging to 2 classes.
Found 500 images belonging to 2 classes.
Found 500 images belonging to 2 classes.
Epoch 1/25
75/75 [==============================] - 18s 222ms/step - loss: 0.7599 - accuracy: 0.5108 - val_loss: 0.6964 - val_accuracy: 0.4980
Epoch 2/25
75/75 [==============================] - 16s 215ms/step - loss: 0.6778 - accuracy: 0.5767 - val_loss: 0.6385 - val_accuracy: 0.6360
Epoch 3/25
75/75 [==============================] - 16s 214ms/step - loss: 0.6317 - accuracy: 0.6558 - val_loss: 0.5621 - val_accuracy: 0.7320
Epoch 4/25
75/75 [==============================] - 16s 213ms/step - loss: 0.5517 - accuracy: 0.7175 - val_loss: 0.4531 - val_accuracy: 0.8020
Epoch 5/25
75/75 [==============================] - 16s 210ms/step - loss: 0.4566 - accuracy: 0.7921 - val_loss: 0.4020 - val_accuracy: 0.7960
Epoch 6/25
75/75 [==============================] - 16s 213ms/step - loss: 0.3512 - accuracy: 0.8429 - val_loss: 0.2472 - val_accuracy: 0.9080
Epoch 7/25
75/75 [

In this task, I increased the training sample size to 1200 images per class (2400 total) and retrained a CNN model from scratch to improve performance on the Cats & Dogs classification task. Here’s a summary of each approach and the resulting performance:

Baseline Model: With a larger training dataset (2400 images) and without any optimizations, the CNN model achieved a perfect test accuracy of 100%, indicating the potential for overfitting due to the increased sample size.

Optimized Model: I introduced data augmentation techniques (rotation, shift, shear, zoom, and horizontal flip) to increase variability and prevent overfitting. Additionally, I added dropout and batch normalization layers to improve generalization and training stability. This optimized model achieved a test accuracy of 76.2%.

By increasing the sample size and implementing data augmentation, dropout, and batch normalization, I successfully improved the model's robustness and balanced its generalization, albeit with some reduction in accuracy due to reduced overfitting tendencies. The final optimized model yielded a realistic performance of 76.2% test accuracy, reflecting a more generalizable approach compared to the overfitted baseline.

In [ ]:
#Question - 3
import os
import shutil
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Helper function to prepare the dataset with a specified training sample size
def prepare_dataset(train_size_per_class):
    """
    This function prepares the dataset by adjusting the number of training images per class
    to the specified size, while keeping validation and test sets the same.
    """
    # Set up paths
    base_dir = '/content/cats_and_dogs'
    train_dir = os.path.join(base_dir, 'train')
    validation_dir = os.path.join(base_dir, 'validation')
    test_dir = os.path.join(base_dir, 'test')
    source_dir = 'train'  # Directory with all images

    # Clear and recreate training directories if they exist
    if os.path.exists(train_dir):
        shutil.rmtree(train_dir)
    os.makedirs(train_dir + '/cats', exist_ok=True)
    os.makedirs(train_dir + '/dogs', exist_ok=True)

    # Get lists of all images
    all_images = os.listdir(source_dir)
    cat_images = [img for img in all_images if 'cat' in img]
    dog_images = [img for img in all_images if 'dog' in img]

    # Function to copy images to the target directory
    def copy_images(images, start, end, target_dir):
        for img in images[start:end]:
            shutil.copy(os.path.join(source_dir, img), target_dir)

    # Copy images for specified training size per class
    copy_images(cat_images, 0, train_size_per_class, os.path.join(train_dir, 'cats'))
    copy_images(dog_images, 0, train_size_per_class, os.path.join(train_dir, 'dogs'))

# Helper function to define, train, and evaluate the model
def train_and_evaluate_model(train_size_per_class):
    """
    This function defines the CNN model with data augmentation, dropout, and batch normalization,
    trains it with the specified training sample size, and evaluates it on the test set.
    """
    # Prepare the dataset for the specified training sample size
    prepare_dataset(train_size_per_class)

    # Data Augmentation for training set
    train_datagen_augmented = ImageDataGenerator(
        rescale=1./255,
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest'
    )

    # Rescale only for validation and test sets
    validation_datagen = ImageDataGenerator(rescale=1./255)
    test_datagen = ImageDataGenerator(rescale=1./255)

    # Data Generators
    train_generator_augmented = train_datagen_augmented.flow_from_directory(
        train_dir,
        target_size=(150, 150),
        batch_size=32,
        class_mode='binary'
    )

    validation_generator = validation_datagen.flow_from_directory(
        validation_dir,
        target_size=(150, 150),
        batch_size=32,
        class_mode='binary'
    )

    test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(150, 150),
        batch_size=32,
        class_mode='binary'
    )

    # Define the CNN Model with Dropout and Batch Normalization
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
        BatchNormalization(),
        MaxPooling2D(2, 2),

        Conv2D(64, (3, 3), activation='relu'),
        BatchNormalization(),
        MaxPooling2D(2, 2),

        Conv2D(128, (3, 3), activation='relu'),
        BatchNormalization(),
        MaxPooling2D(2, 2),

        Flatten(),
        Dense(512, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid')
    ])

    # Compile the model
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    # Train the model
    history = model.fit(
        train_generator_augmented,
        steps_per_epoch=len(train_generator_augmented),
        epochs=25,  # More epochs to fully utilize each dataset size
        validation_data=validation_generator,
        validation_steps=len(validation_generator)
    )

    # Evaluate the model on test data
    test_loss, test_acc = model.evaluate(test_generator, steps=len(test_generator))
    print(f"Test accuracy with training size ({train_size_per_class * 2} total images): {test_acc}")
    return test_acc

# Experiment with different training sizes and record the results
sample_sizes = [300, 600, 1500, 3000]  # Experiment with sizes smaller and larger than before
results = {}

for size in sample_sizes:
    print(f"\nTraining with sample size: {size * 2} (Total images: {size * 2})")
    results[size * 2] = train_and_evaluate_model(size)

# Display the results for comparison
print("\nResults for Different Training Sizes:")
for size, accuracy in results.items():
    print(f"Training Size: {size} images, Test Accuracy: {accuracy}")



Training with sample size: 600 (Total images: 600)
Found 600 images belonging to 2 classes.
Found 500 images belonging to 2 classes.
Found 500 images belonging to 2 classes.
Epoch 1/25
19/19 [==============================] - 8s 362ms/step - loss: 7.9478 - accuracy: 0.5467 - val_loss: 4.6508 - val_accuracy: 0.5000
Epoch 2/25
19/19 [==============================] - 7s 343ms/step - loss: 3.6523 - accuracy: 0.5717 - val_loss: 4.0907 - val_accuracy: 0.5280
Epoch 3/25
19/19 [==============================] - 7s 345ms/step - loss: 2.1038 - accuracy: 0.5717 - val_loss: 6.6496 - val_accuracy: 0.5000
Epoch 4/25
19/19 [==============================] - 7s 341ms/step - loss: 1.2115 - accuracy: 0.6317 - val_loss: 13.3004 - val_accuracy: 0.5000
Epoch 5/25
19/19 [==============================] - 7s 341ms/step - loss: 0.8980 - accuracy: 0.6300 - val_loss: 8.7373 - val_accuracy: 0.5020
Epoch 6/25
19/19 [==============================] - 7s 344ms/step - loss: 0.8198 - accuracy: 0.6033 - val_loss: 8.

To determine the optimal training sample size for the Cats & Dogs classification task, I trained a model with increasing training sample sizes: 600, 1200, 3000, and 6000 images, while keeping validation and test sets constant. I applied data augmentation, dropout, and batch normalization in the model to improve performance and reduce overfitting.

The test accuracies achieved with each training size were as follows:

600 images: 68.0% test accuracy
1200 images: 69.8% test accuracy
3000 images: 72.0% test accuracy
6000 images: 80.2% test accuracy
With the largest sample size of 6000 images, the model achieved the best test accuracy of 80.2%, suggesting that a higher training sample size positively impacts model performance in this task.

In [ ]:
#Questiopn-04

import os
import shutil
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau

# Helper function to prepare the dataset with a specified training sample size
def prepare_dataset(train_size_per_class):
    """
    This function prepares the dataset by adjusting the number of training images per class
    to the specified size, while keeping validation and test sets the same.
    """
    # Set up paths
    base_dir = '/content/cats_and_dogs'
    train_dir = os.path.join(base_dir, 'train')
    validation_dir = os.path.join(base_dir, 'validation')
    test_dir = os.path.join(base_dir, 'test')
    source_dir = 'train'  # Directory with all images

    # Clear and recreate training directories if they exist
    if os.path.exists(train_dir):
        shutil.rmtree(train_dir)
    os.makedirs(train_dir + '/cats', exist_ok=True)
    os.makedirs(train_dir + '/dogs', exist_ok=True)

    # Get lists of all images
    all_images = os.listdir(source_dir)
    cat_images = [img for img in all_images if 'cat' in img]
    dog_images = [img for img in all_images if 'dog' in img]

    # Function to copy images to the target directory
    def copy_images(images, start, end, target_dir):
        for img in images[start:end]:
            shutil.copy(os.path.join(source_dir, img), target_dir)

    # Copy images for specified training size per class
    copy_images(cat_images, 0, train_size_per_class, os.path.join(train_dir, 'cats'))
    copy_images(dog_images, 0, train_size_per_class, os.path.join(train_dir, 'dogs'))

# Helper function to define, train, and evaluate the model with a pretrained base
def train_and_evaluate_model_with_pretrained(train_size_per_class):
    """
    This function defines a CNN model with a pretrained VGG16 base, additional dense layers, and
    optimizations. It trains the model with the specified training sample size and evaluates it on the test set.
    """
    # Prepare the dataset for the specified training sample size
    prepare_dataset(train_size_per_class)

    # Data Augmentation for training set
    train_datagen_augmented = ImageDataGenerator(
        rescale=1./255,
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest'
    )

    # Rescale only for validation and test sets
    validation_datagen = ImageDataGenerator(rescale=1./255)
    test_datagen = ImageDataGenerator(rescale=1./255)

    # Data Generators
    train_generator_augmented = train_datagen_augmented.flow_from_directory(
        train_dir,
        target_size=(150, 150),
        batch_size=32,
        class_mode='binary'
    )

    validation_generator = validation_datagen.flow_from_directory(
        validation_dir,
        target_size=(150, 150),
        batch_size=32,
        class_mode='binary'
    )

    test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(150, 150),
        batch_size=32,
        class_mode='binary'
    )

    # Define the pretrained VGG16 base model
    vgg16_base = VGG16(weights='imagenet', include_top=False, input_shape=(150, 150, 3))

    # Freeze the base model layers initially
    vgg16_base.trainable = False

    # Define the model architecture with the pretrained base and custom top layers
    model = Sequential([
        vgg16_base,
        Flatten(),
        Dense(512, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),
        Dense(1, activation='sigmoid')  # Sigmoid for binary classification
    ])

    # Compile the model with a lower learning rate
    model.compile(optimizer=Adam(learning_rate=1e-4), loss='binary_crossentropy', metrics=['accuracy'])

    # Add a learning rate scheduler
    lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)

    # Train the model with augmented data
    history = model.fit(
        train_generator_augmented,
        steps_per_epoch=len(train_generator_augmented),
        epochs=25,  # Use more epochs to fully utilize each dataset size
        validation_data=validation_generator,
        validation_steps=len(validation_generator),
        callbacks=[lr_scheduler]
    )

    # Unfreeze the last few layers of the base model for fine-tuning
    for layer in vgg16_base.layers[-4:]:
        layer.trainable = True

    # Recompile the model with a lower learning rate for fine-tuning
    model.compile(optimizer=Adam(learning_rate=1e-5), loss='binary_crossentropy', metrics=['accuracy'])

    # Fine-tune the model
    history_finetune = model.fit(
        train_generator_augmented,
        steps_per_epoch=len(train_generator_augmented),
        epochs=25,  # Fine-tune for fewer epochs
        validation_data=validation_generator,
        validation_steps=len(validation_generator),
        callbacks=[lr_scheduler]
    )

    # Evaluate the model on test data
    test_loss, test_acc = model.evaluate(test_generator, steps=len(test_generator))
    print(f"Test accuracy with pretrained model and training size ({train_size_per_class * 2} total images): {test_acc}")
    return test_acc

# Experiment with different training sizes and record the results
sample_sizes = [300, 600, 1500, 3000]  # Experiment with sizes smaller and larger than before
results = {}

for size in sample_sizes:
    print(f"\nTraining with pretrained model and sample size: {size * 2} (Total images: {size * 2})")
    results[size * 2] = train_and_evaluate_model_with_pretrained(size)

# Display the results for comparison
print("\nResults for Different Training Sizes (Pretrained Network):")
for size, accuracy in results.items():
    print(f"Training Size: {size} images, Test Accuracy: {accuracy}")



Training with pretrained model and sample size: 600 (Total images: 600)
Found 600 images belonging to 2 classes.
Found 500 images belonging to 2 classes.
Found 500 images belonging to 2 classes.
58889256/58889256 [==============================] - 0s 0us/step
Epoch 1/25
19/19 [==============================] - 13s 605ms/step - loss: 0.7506 - accuracy: 0.6433 - val_loss: 0.4955 - val_accuracy: 0.7520 - lr: 1.0000e-04
Epoch 2/25
19/19 [==============================] - 11s 562ms/step - loss: 0.4886 - accuracy: 0.7650 - val_loss: 0.4308 - val_accuracy: 0.7880 - lr: 1.0000e-04
Epoch 3/25
19/19 [==============================] - 11s 560ms/step - loss: 0.4846 - accuracy: 0.8017 - val_loss: 0.3744 - val_accuracy: 0.8420 - lr: 1.0000e-04
Epoch 4/25
19/19 [==============================] - 11s 568ms/step - loss: 0.3740 - accuracy: 0.8183 - val_loss: 0.3307 - val_accuracy: 0.8680 - lr: 1.0000e-04
Epoch 5/25
19/19 [==============================] - 11s 562ms/step - loss: 0.3542 - accuracy: 0.846

Using the pretrained VGG16 model, I tested various sample sizes to evaluate their effect on model performance. Here’s an expanded breakdown of the results and observations:

600 Images (Total):
With this small sample size, the pretrained VGG16 model reached an impressive 88.4% accuracy on the test set. This is significantly better than the accuracy achieved using a model trained from scratch with the same sample size, indicating that the pretrained model could generalize well even with limited data. Transfer learning allowed the model to leverage features learned from a large dataset (ImageNet), giving it a strong starting point even with fewer training images.

1200 Images (Total):
Doubling the sample size to 1200 images further improved performance, yielding a 90.0% accuracy on the test set. This accuracy increase suggests that the pretrained model could still benefit from additional data, enabling it to better adapt to the specific features of the Cats and Dogs dataset.

3000 Images (Total):
Using 3000 images provided a substantial improvement, leading to a peak accuracy of 94.2% on the test set. This was the highest accuracy achieved across all sample sizes with the pretrained model, indicating that 3000 images might be an optimal balance for this dataset. At this size, the model had sufficient data to fine-tune its weights effectively while still leveraging the pretrained layers' representations.

6000 Images (Total):
With the largest sample size, the pretrained model achieved 93.0% accuracy, which was slightly lower than with 3000 images. This result could be due to several factors, such as overfitting or reaching a plateau in model performance given the dataset characteristics. While 6000 images still provided high accuracy, it did not significantly improve over the 3000-image training set.

Overall Observations:
The pretrained model provided better performance across all sample sizes compared to training a model from scratch. The optimal test accuracy of 94.2% was achieved with a training sample size of 3000 images, suggesting that this sample size effectively balanced model complexity and available data. Pretrained models can greatly enhance performance on smaller datasets, as they retain useful features learned from large, diverse datasets, thus reducing the amount of data required to achieve high accuracy in new tasks